In [1]:
from __future__ import annotations
import math
import os
import zipfile
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional

import cv2
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# ----------------------------
# Data structures
# ----------------------------

@dataclass
class FittedCircle:
    cx: float
    cy: float
    r: float

@dataclass
class Defect:
    boundary: str  # 'outer' or 'inner'
    kind: str      # 'flash' or 'cut'
    angle_deg: float
    arc_deg: float
    size_px: float
    bbox: Tuple[int, int, int, int]  # x, y, w, h
    centroid: Tuple[float, float]

In [3]:
# ----------------------------
# Utility
# ----------------------------

_DEF_FIGSIZE = (6, 6)

def _show(title: str, img: np.ndarray) -> None:
    plt.figure(figsize=_DEF_FIGSIZE)
    if img.ndim == 2:
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()


In [4]:
# ----------------------------
# 1) Preprocessing & masking
# ----------------------------

def binarize_ring(image: np.ndarray) -> np.ndarray:
    if image.ndim == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()

    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    def foreground_score(mask: np.ndarray) -> float:
        h, w = mask.shape
        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
        best = 0.0
        for i in range(1, num_labels):
            x, y, ww, hh, area = stats[i]
            touches = (x == 0 or y == 0 or x + ww == w or y + hh == h)
            if not touches:
                best = max(best, float(area))
        return best

    mask = th if foreground_score(th) >= foreground_score(255 - th) else (255 - th)

    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    if num_labels > 1:
        largest = 1 + int(np.argmax(stats[1:, cv2.CC_STAT_AREA]))
        mask = np.where(labels == largest, 255, 0).astype(np.uint8)

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    ring = (mask > 0).astype(np.uint8) * 255
    return ring

In [5]:
# ----------------------------
# 2) Contours & fitting
# ----------------------------

def get_ring_contours(ring_mask: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    contours, hierarchy = cv2.findContours(ring_mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    if not contours or hierarchy is None:
        raise ValueError("No contours found in ring mask.")

    hierarchy = hierarchy[0]
    areas = [cv2.contourArea(c) for c in contours]

    outer_idx = int(np.argmax(areas))
    child_indices = [i for i, h in enumerate(hierarchy) if h[3] == outer_idx]

    if not child_indices:
        sorted_idx = np.argsort(areas)[::-1]
        inner_idx = int(sorted_idx[1]) if len(sorted_idx) > 1 else outer_idx
    else:
        inner_idx = int(sorted(child_indices, key=lambda i: areas[i])[-1])

    outer = contours[outer_idx]
    inner = contours[inner_idx]
    return outer, inner

def fit_circle_least_squares(contour: np.ndarray) -> FittedCircle:
    pts = contour.reshape(-1, 2).astype(np.float64)
    x = pts[:, 0]
    y = pts[:, 1]
    x_m = np.mean(x)
    y_m = np.mean(y)

    u = x - x_m
    v = y - y_m
    Suu = np.dot(u, u)
    Svv = np.dot(v, v)
    Suv = np.dot(u, v)
    Suuu = np.dot(u, u * u)
    Svvv = np.dot(v, v * v)
    Suvv = np.dot(u, v * v)
    Svuu = np.dot(v, u * u)

    A = np.array([[Suu, Suv], [Suv, Svv]], dtype=np.float64)
    b = 0.5 * np.array([Suuu + Suvv, Svvv + Svuu], dtype=np.float64)
    try:
        uc, vc = np.linalg.solve(A, b)
    except np.linalg.LinAlgError:
        (cx, cy), r = cv2.minEnclosingCircle(contour)
        return FittedCircle(float(cx), float(cy), float(r))

    cx = x_m + uc
    cy = y_m + vc
    r = np.mean(np.sqrt((x - cx) ** 2 + (y - cy) ** 2))
    return FittedCircle(float(cx), float(cy), float(r))

In [6]:
# ----------------------------
# 3) Deviation analysis & defect detection
# ----------------------------

def _moving_average(x: np.ndarray, k: int) -> np.ndarray:
    if k <= 1:
        return x
    k = int(max(1, k))
    pad = k // 2
    xpad = np.pad(x, (pad, pad), mode='wrap')
    c = np.convolve(xpad, np.ones(k) / k, mode='valid')
    return c[: x.shape[0]]

def analyze_contour(contour: np.ndarray, circle: FittedCircle, smooth_window: int = 5) -> Dict[str, np.ndarray]:
    pts = contour.reshape(-1, 2).astype(np.float64)
    x = pts[:, 0]
    y = pts[:, 1]
    theta = np.mod(np.arctan2(y - circle.cy, x - circle.cx), 2 * np.pi)
    r = np.sqrt((x - circle.cx) ** 2 + (y - circle.cy) ** 2)
    order = np.argsort(theta)
    theta = theta[order]
    r = r[order]
    xy_sorted = pts[order]
    delta = r - circle.r
    delta = _moving_average(delta, smooth_window)
    return {"theta": theta, "r": r, "delta": delta, "xy_sorted": xy_sorted}

def _mad(x: np.ndarray) -> float:
    med = np.median(x)
    return float(np.median(np.abs(x - med)) + 1e-9)

def group_runs(mask: np.ndarray, theta: np.ndarray, min_arc_deg: float = 0.4) -> List[Tuple[int, int]]:
    n = mask.size
    if n == 0:
        return []
    mask2 = np.concatenate([mask, mask])
    theta2 = np.concatenate([theta, theta + 2 * np.pi])
    runs = []
    i = 0
    while i < mask2.size:
        if not mask2[i]:
            i += 1
            continue
        j = i
        while j < mask2.size and mask2[j]:
            j += 1
        arc = (theta2[j - 1] - theta2[i]) * 180 / np.pi
        if arc >= min_arc_deg:
            runs.append((i, j - 1))
        i = j
    mapped: List[Tuple[int, int]] = []
    for (a, b) in runs:
        a %= n
        b %= n
        if a <= b:
            mapped.append((a, b))
        else:
            mapped.append((a, n - 1))
            mapped.append((0, b))
    mapped.sort()
    merged: List[Tuple[int, int]] = []
    for seg in mapped:
        if not merged or seg[0] > merged[-1][1] + 1:
            merged.append(list(seg))
        else:
            merged[-1][1] = max(merged[-1][1], seg[1])
    return [(int(a), int(b)) for a, b in merged]

def detect_defects_on_boundary(boundary_name: str, contour: np.ndarray, circle: FittedCircle, min_arc_deg: float = 0.4) -> List[Defect]:
    ana = analyze_contour(contour, circle)
    theta = ana["theta"]
    delta = ana["delta"]
    pts = ana["xy_sorted"]
    mad = _mad(delta)
    thr = max(2.5 * mad, 0.003 * circle.r, 0.75)
    mask = np.abs(delta) > thr
    segments = group_runs(mask, theta, min_arc_deg=min_arc_deg)
    defects: List[Defect] = []
    for a, b in segments:
        seg_pts = pts[a : b + 1]
        seg_theta = theta[a : b + 1]
        seg_delta = delta[a : b + 1]
        mean_delta = float(np.mean(seg_delta))
        ang_mid = float((seg_theta[0] + seg_theta[-1]) / 2.0)
        arc_deg = float((seg_theta[-1] - seg_theta[0]) * 180 / np.pi)
        xs = seg_pts[:, 0]
        ys = seg_pts[:, 1]
        x0, y0, x1, y1 = int(xs.min()), int(ys.min()), int(xs.max()), int(ys.max())
        bbox = (x0, y0, x1 - x0 + 1, y1 - y0 + 1)
        centroid = (float(xs.mean()), float(ys.mean()))
        if boundary_name == 'outer':
            kind = 'flash' if mean_delta > 0 else 'cut'
        else:
            kind = 'flash' if mean_delta < 0 else 'cut'
        size_px = float(np.max(np.abs(seg_delta)))
        defects.append(
            Defect(boundary=boundary_name, kind=kind, angle_deg=(ang_mid * 180 / np.pi) % 360.0,
                   arc_deg=arc_deg, size_px=size_px, bbox=bbox, centroid=centroid)
        )
    return defects


In [7]:
# ----------------------------
# 4) Public API per image
# ----------------------------

def analyze_image(image: np.ndarray, debug: bool = False) -> Tuple[np.ndarray, List[Defect], Dict[str, FittedCircle]]:
    ring = binarize_ring(image)
    outer, inner = get_ring_contours(ring)
    c_outer = fit_circle_least_squares(outer)
    c_inner = fit_circle_least_squares(inner)
    defects = []
    defects += detect_defects_on_boundary('outer', outer, c_outer)
    defects += detect_defects_on_boundary('inner', inner, c_inner)
    overlay = cv2.cvtColor((ring > 0).astype(np.uint8) * 255, cv2.COLOR_GRAY2BGR)

    def draw_circle(img, c: FittedCircle, color=(0, 255, 0)):
        cv2.circle(img, (int(round(c.cx)), int(round(c.cy))), int(round(c.r)), color, 2)
        cv2.circle(img, (int(round(c.cx)), int(round(c.cy))), 3, (0, 0, 255), -1)

    draw_circle(overlay, c_outer, (0, 255, 0))
    draw_circle(overlay, c_inner, (0, 255, 0))

    for d in defects:
        x, y, w, h = d.bbox
        color = (0, 0, 255) if d.kind == 'cut' else (255, 0, 0)
        cv2.rectangle(overlay, (x, y), (x + w, y + h), color, 2)
        label = f"{d.boundary[:1].upper()}-{d.kind} {d.angle_deg:.1f}°"
        cv2.putText(overlay, label, (x, max(0, y - 6)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1, cv2.LINE_AA)

    circles = {"outer": c_outer, "inner": c_inner}
    return overlay, defects, circles

In [10]:
# ----------------------------
# 5) Batch runner & saving to ZIP
# ----------------------------

def run_and_save(paths: List[str], output_dir: str = "output", zip_name: str = "results.zip"):
    os.makedirs(output_dir, exist_ok=True)
    for p in paths:
        img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"[WARN] Could not read: {p}")
            continue
        overlay, defects, circles = analyze_image(img)
        out_path = os.path.join(output_dir, os.path.basename(p))
        cv2.imwrite(out_path, overlay)
        print(f"Saved: {out_path}")

    # Zip all results
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        for file_name in os.listdir(output_dir):
            file_path = os.path.join(output_dir, file_name)
            zipf.write(file_path, arcname=file_name)
    print(f"Zipped results to: {zip_name}")

In [11]:

# ----------------------------
# 6) Main execution
# ----------------------------
if __name__ == "__main__":
    IMAGE_PATHS = [
        "defect1.png",
        "defect2.png",
        "defect3.png",
        "defect4.png",
        "good.png",
    ]
    run_and_save(IMAGE_PATHS)

Saved: output\defect1.png
Saved: output\defect2.png
Saved: output\defect3.png
Saved: output\defect4.png
Saved: output\good.png
Zipped results to: results.zip
